The purpose of this notebook is to create a function that takes the imported player list:
- clean to only needed information
- seperate by pither and batter
- merge each on player name to get projections from model
- merge back pitcher and batter data 
- output df for now

Next steps:
- build line up based on salary

In [3]:
# imports
import pandas as pd
import numpy as np

In [4]:
# will need fanduel import
fd = pd.read_csv('../../../Downloads/FanDuel-MLB-2021 ET-05 ET-11 ET-58668-players-list.csv')

In [5]:
# print first 5 rows of fd
fd.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Probable Pitcher,Batting Order,Roster Position
0,58668-52859,P,Jacob,Jacob deGrom,deGrom,57.333333,6.0,12500,BAL@NYM,NYM,BAL,O,Side,NaN,NaN,0.0,P
1,58668-16956,P,Gerrit,Gerrit Cole,Cole,50.857143,7.0,12300,NYY@TB,NYY,TB,NaN,NaN,NaN,NaN,NaN,P
2,58668-5481,P,Max,Max Scherzer,Scherzer,45.428571,7.0,12000,PHI@WSH,WSH,PHI,NaN,NaN,NaN,NaN,0.0,P
3,58668-52183,P,Tyler,Tyler Glasnow,Glasnow,47.250000,8.0,11500,NYY@TB,TB,NYY,NaN,NaN,NaN,NaN,0.0,P
4,58668-60633,P,Carlos,Carlos Rodon,Rodon,52.200000,5.0,11200,MIN@CWS,CWS,MIN,NaN,NaN,NaN,NaN,0.0,P


only columns needed:
- id - will need this later for template
- Position
- nickname - renamed to Name
- salary
- game
- team
- opponent
- injury indicator
- probable pitcher

In [6]:
fd.columns

Index(['Id', 'Position', 'First Name', 'Nickname', 'Last Name', 'FPPG',
       'Played', 'Salary', 'Game', 'Team', 'Opponent', 'Injury Indicator',
       'Injury Details', 'Tier', 'Probable Pitcher', 'Batting Order',
       'Roster Position'],
      dtype='object')

In [7]:
# clean up fd to match column list above
fd.drop(columns=['First Name', 'Last Name', 'FPPG', 'Played',
                 'Injury Details', 'Tier', 'Batting Order', 'Roster Position'], inplace=True)

Next step is fill nulls in probable pitcher and injury indicator

In [8]:
# filling nulls for probable pitcher
fd['Probable Pitcher'].fillna('No', inplace=True)

In [9]:
# fill nulls for injury indicator
fd['Injury Indicator'].fillna('Healthy', inplace=True)

In [10]:
# review new cleaned df
fd.head()

,Id,Position,Nickname,Salary,Game,Team,Opponent,Injury Indicator,Probable Pitcher
0,58668-52859,P,Jacob deGrom,12500,BAL@NYM,NYM,BAL,O,No
1,58668-16956,P,Gerrit Cole,12300,NYY@TB,NYY,TB,Healthy,No
2,58668-5481,P,Max Scherzer,12000,PHI@WSH,WSH,PHI,Healthy,No
3,58668-52183,P,Tyler Glasnow,11500,NYY@TB,TB,NYY,Healthy,No
4,58668-60633,P,Carlos Rodon,11200,MIN@CWS,CWS,MIN,Healthy,No


In [11]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939 entries, 0 to 938
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                939 non-null    object
 1   Position          939 non-null    object
 2   Nickname          939 non-null    object
 3   Salary            939 non-null    int64 
 4   Game              939 non-null    object
 5   Team              939 non-null    object
 6   Opponent          939 non-null    object
 7   Injury Indicator  939 non-null    object
 8   Probable Pitcher  939 non-null    object
dtypes: int64(1), object(8)
memory usage: 66.1+ KB


Next step rename Nickname to name

In [12]:
# renaming nickname column
fd.rename(columns={'Nickname': 'Name'}, inplace=True)

Next step filter to only healthy players

In [13]:
# fitler to only healthy players
fd = fd.loc[fd['Injury Indicator'] == 'Healthy']

In [14]:
# review dataframe
fd.head()

,Id,Position,Name,Salary,Game,Team,Opponent,Injury Indicator,Probable Pitcher
1,58668-16956,P,Gerrit Cole,12300,NYY@TB,NYY,TB,Healthy,No
2,58668-5481,P,Max Scherzer,12000,PHI@WSH,WSH,PHI,Healthy,No
3,58668-52183,P,Tyler Glasnow,11500,NYY@TB,TB,NYY,Healthy,No
4,58668-60633,P,Carlos Rodon,11200,MIN@CWS,CWS,MIN,Healthy,No
6,58668-16959,P,Trevor Bauer,11000,SEA@LAD,LAD,SEA,Healthy,No


In [15]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 755 entries, 1 to 938
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                755 non-null    object
 1   Position          755 non-null    object
 2   Name              755 non-null    object
 3   Salary            755 non-null    int64 
 4   Game              755 non-null    object
 5   Team              755 non-null    object
 6   Opponent          755 non-null    object
 7   Injury Indicator  755 non-null    object
 8   Probable Pitcher  755 non-null    object
dtypes: int64(1), object(8)
memory usage: 59.0+ KB


next step is to split into pithcers and batters

In [16]:
# split using .loc by position and make new dataframe for pitchers
pitchers = fd.loc[fd['Position']=='P']

In [17]:
# split using .loc by position and make new dataframe for batters
batters = fd.loc[fd['Position']!='P']

Now we have cleaned dataframes for each group. Now lets focus on pitchers to get projections added to dataframe.

First step for pitchers is to get the probable pitchers first, we only care about the ones that will start.

In [18]:
# save pitchers df to only starting pitchers
pitchers = pitchers.loc[pitchers['Probable Pitcher']=='Yes']

In [19]:
# review changes
pitchers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 7 to 179
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                22 non-null     object
 1   Position          22 non-null     object
 2   Name              22 non-null     object
 3   Salary            22 non-null     int64 
 4   Game              22 non-null     object
 5   Team              22 non-null     object
 6   Opponent          22 non-null     object
 7   Injury Indicator  22 non-null     object
 8   Probable Pitcher  22 non-null     object
dtypes: int64(1), object(8)
memory usage: 1.7+ KB


Next step is to combine projections with 2021 stats.
- import testing data with projections from model
- merge the two data frames

In [20]:
# read in pitcher projections
pitcher_proj = pd.read_csv('../Projections/pitcher_projections_2021.csv')

Next step merge.

In [21]:
# merge attempt
pitcher_projections = pitchers.merge(pitcher_proj, how='left', on='Name')

Next step is to drop nulls from this list.  Will need to handle this in previous processes for final product.

In [22]:
pitcher_projections.dropna(inplace=True)

Next step is to clean up and only take what is needed from pitchers so that it can be merged with batters.
Columns needed:
- ID
- Position
- Name
- Salary
- Team
- Opponent
- Proj_FPPG

In [23]:
# overwrite df with only the columns needed
pitcher_projections = pitcher_projections[['Id', 'Position', 'Name', 'Salary', 'Team_x', 'Opponent', 'AVG', 'Projected_FPPG']]

In [24]:
# rename team column
pitcher_projections.rename(columns={'Team_x' : 'Team'}, inplace=True)

Now we have a cleaned and organized df of our pitchers that are starting. Next is the batters.

---

First step is to merge on projections.

In [25]:
# read in projections file
batter_21 = pd.read_csv('../Projections/batter_projections_2021.csv')

In [26]:
# merge projections with batter df, creating new df
batters_projections = batters.merge(batter_21, how='left', on='Name')

Will have to drop nulls for now, but need to figure out why there is no data for those batters.

In [27]:
# dropping batters with no projections
batters_projections.dropna(inplace=True)

In [28]:
batters_projections

,Id,Position,Name,Salary,Game,Team_x,Opponent,Injury Indicator,Probable Pitcher,Team_y,...,CS,TB,AVG,OBP,SLG,OPS,ISO,PTS,FPPG,Projected_FPPG
0,58668-79393,SS,Fernando Tatis Jr.,4700,SD@COL,SD,COL,Healthy,No,SD,...,1.0,53.0,0.240,0.315,0.552,0.867,0.312,356.6,13.715385,9.161655
2,58668-12933,OF,Mike Trout,4400,LAA@HOU,LAA,HOU,Healthy,No,LAA,...,0.0,72.0,0.365,0.484,0.692,1.176,0.327,433.4,13.980645,11.217342
3,58668-82575,OF,Yordan Alvarez,4300,LAA@HOU,HOU,LAA,Healthy,No,HOU,...,0.0,67.0,0.345,0.378,0.609,0.987,0.264,362.7,12.953571,9.453620
4,58668-13540,OF,J.D. Martinez,4200,OAK@BOS,BOS,OAK,Healthy,No,BOS,...,0.0,84.0,0.338,0.427,0.646,1.073,0.308,519.7,15.285294,10.858018
5,58668-39151,OF,Mookie Betts,4100,SEA@LAD,LAD,SEA,Healthy,No,LAD,...,2.0,51.0,0.261,0.380,0.443,0.823,0.182,342.3,12.225000,8.155783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,58668-81869,C,Chad Wallach,2000,MIA@ARI,MIA,ARI,Healthy,No,MIA,...,0.0,16.0,0.255,0.275,0.340,0.615,0.085,77.9,4.582353,5.035409
349,58668-12237,C,Jonathan Lucroy,2000,PHI@WSH,WSH,PHI,Healthy,No,WSH,...,0.0,6.0,0.357,0.357,0.429,0.786,0.072,25.0,5.000000,6.212545
350,58668-102358,SS,Jose Devers,2000,MIA@ARI,MIA,ARI,Healthy,No,MIA,...,0.0,2.0,0.167,0.154,0.167,0.321,0.000,15.9,1.987500,2.197133
351,58668-102363,OF,Luke Raley,2000,SEA@LAD,LAD,SEA,Healthy,No,LAD,...,0.0,11.0,0.206,0.308,0.324,0.632,0.118,61.4,4.385714,5.508057


In [29]:
# drop unneeded columns for merge with pitcher, overwrite current df
batters_projections = batters_projections[['Id', 'Position', 'Name', 'Salary', 'Team_x', 'Opponent','Projected_FPPG']]

In [30]:
# rename team column
batters_projections.rename(columns={'Team_x' : 'Team'}, inplace=True)

C:\Users\phill\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Now there are two clean dataframes with the same columns, now we will determine optimal line up.

Steps to get optimal line up:
1. set salary cap
2. sort by fppg

    a. choose pitcher - based on projected fppg; in final product need to incoporate stats with strong coefficient values
    b. choose batters based on remaining salary and highest fppg; in final product need to incoporate stats with strong coefficient values
    
3. create list to append choices to
4. save to one df for output
5. take df and save to templet for export

In [31]:
# set cap for fanduel
salary_cap = 35_000

In [32]:
# sort by avg
pitcher_projections.sort_values(by='AVG', ascending=False, inplace=True, ignore_index=True)

In [33]:
# create list of teams to filter batters
team_list = []
for x in range(0,4):
    team_list.append(pitcher_projections['Opponent'][x])

In [34]:
# drop avg
pitcher_projections.drop(columns='AVG', inplace=True)

In [35]:
# create team filter 
team_filter = (batters_projections['Team'] == team_list[0]) | (batters_projections['Team'] == team_list[1]) |(batters_projections['Team'] == team_list[2]) |(batters_projections['Team'] == team_list[3])
# new batter dataframe with team filter
batters_projections = batters_projections[team_filter]
# reset index
batters_projections.reset_index(drop=True, inplace=True)

In [36]:
# clean position
pos_list = [pos[:2] for pos in batters_projections['Position']]
batters_projections['Position'] = pos_list

In [37]:
# sort pitcher by fppg projections
pitcher_projections.sort_values(by='Projected_FPPG', ascending=False, inplace=True, ignore_index=True)

In [38]:
# create a player list to 
lineup = []
lineup.append(pitcher_projections.values[0])

In [39]:
# need to update remaining salary
salary_cap -= pitcher_projections['Salary'][0]

In [40]:
# with updated salary fill remaining roster based on position and highest fppg
# create position list for remaining roster spots
position_list = ['C', '1B', '2B', '3B', 'SS', 'OF', 'OF', 'OF']
# sort batters by FPPG
batters_projections.sort_values(by='Projected_FPPG', ascending=False, inplace=True, ignore_index=True)

In [41]:
# redo 
# create count based on remaining positions
sal_count = 8

# create average salary variable for remaining players
avg_sal = salary_cap/sal_count

# create for loop for each position in list to take highest fppg
for pos in position_list:
    # setting counter to increase if player is already in list
    # this is inside the for loop beacuse it needs to be per position
    counter = 0
    # if salary greater than average move to next player
    for salary in batters_projections.loc[batters_projections['Position'] == pos]['Salary']:
        # test if salary is greater than average if it is increase counter
        if batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter] > avg_sal:
            counter += 1
        else:
            # if less than average add player to list
            lineup.append(batters_projections.loc[batters_projections['Position'] == pos].values[counter])
            # drop player so no duplicates are added
            batters_projections.drop(batters_projections.loc[batters_projections['Position'] == pos].index.values[counter], inplace=True)
            # decrease sal_count
            sal_count -= 1
            # decrease salary cap
            salary_cap -= batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter]
            # create new average salary
            avg_sal = salary_cap/sal_count
            break


In [42]:
df = pd.DataFrame(lineup, columns=['Id', 'Position', 'Name', 'Salary', 'Team', 'Opponent','Proj_FPPG'])
df

,Id,Position,Name,Salary,Team,Opponent,Proj_FPPG
0,58668-52159,P,Lance McCullers Jr.,8500,HOU,LAA,40.138767
1,58668-102513,C,Mario Feliciano,2000,MIL,STL,10.944532
2,58668-38571,1B,Travis Shaw,2500,MIL,STL,6.809981
3,58668-21705,2B,Marcus Semien,3400,TOR,ATL,8.272751
4,58668-52158,SS,Trevor Story,3700,COL,SD,8.355613
5,58668-13909,OF,Randal Grichuk,2800,TOR,ATL,8.013935
6,58668-38973,OF,Tyrone Taylor,2400,MIL,STL,7.468444
7,58668-60635,OF,Raimel Tapia,3300,COL,SD,7.334853


In [43]:
df['Salary'].sum()

28600

In [44]:
pd.DataFrame(team_list)

,0
0,TOR
1,COL
2,CWS
3,MIL
